## Connexion à la base de données sur PostgreSQL

_Connexion entre la base de données créée directepment sur PostgreSQL et Python_

---

In [22]:
import os, psycopg2, utils, csv

In [23]:
from pathlib import Path

In [24]:
def ouvrir_connection(nom_bdd, utilisateur, mot_passe, host='localhost', port=5432):
    
    try:
        conn = psycopg2.connect(dbname=nom_bdd, user=utilisateur, password=mot_passe, host=host, port=5432)
    except psycopg2.Error as e:
        print("Erreur lors de la connection à la base de données")
        print(e)
        return None
    conn.set_session(autocommit=True)
    
    return conn

In [25]:
ma_base_donnees = "floupics_mov2"
utilisateur = "postgres"
mot_passe = os.environ.get("pg_psw")

In [26]:
conn = ouvrir_connection(ma_base_donnees, utilisateur, mot_passe)

type(conn)

psycopg2.extensions.connection

---

## Création des tables par le biais de requêtes SQL

_Utilisation de Python pour exécuter des requêtes SQL afin de créer les tables de la BDD_

---

In [27]:
sql_creer_tables = """
    CREATE TABLE IF NOT EXISTS movies (
    movieId INT UNIQUE PRIMARY KEY,
    title TEXT NOT NULL,
    genres TEXT NOT NULL);
    
    CREATE TABLE IF NOT EXISTS links (
    movieId INT PRIMARY KEY NOT NULL,
    imdbId VARCHAR,
    tmdbId VARCHAR,
    FOREIGN KEY (movieId) REFERENCES movies(movieId));
    
    CREATE TABLE IF NOT EXISTS ratings (
    userId INT NOT NULL,
    movieId INT NOT NULL,
    rating REAL NOT NULL,
    timestamp INT NOT NULL,
    PRIMARY KEY (userId, movieId),
    foreign key(movieId) REFERENCES movies(movieId));
    
    CREATE TABLE IF NOT EXISTS tags (
    userId INT NOT NULL,
    movieId INT NOT NULL,
    tag TEXT NOT NULL,
    timestamp INT NOT NULL,
    FOREIGN KEY (movieId) REFERENCES movies(movieId));
"""

In [28]:
def creer_table(conn, sql_creation_table):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_creation_table)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la création de la table")
        print(e)
        return
    cursor.close()
    print("Les tables ont été crées avec succès")

In [29]:
creer_table(conn, sql_creer_tables)

Les tables ont été crées avec succès


---

## Insertion des données par le biais des fichiers CSV

_Insertion des données présentes dans les fichiers CSV fournis dans aa la BDD_

---

In [30]:
import utils

In [31]:
def lire_csv_dict(nom_fichier):
    data = []
    with open(nom_fichier, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

In [32]:
from pathlib import Path

repertoire_data = Path('data/')

chemin_movies = repertoire_data / 'movies.csv'
chemin_links = repertoire_data / 'links.csv'
chemin_ratings = repertoire_data / 'ratings.csv'
chemin_tags = repertoire_data / 'tags.csv'

In [33]:
def inserer_donnees(conn, sql_insertion_table, donnees):
    try:
        cursor = conn.cursor()
        for d in donnees:
            cursor.execute(sql_insertion_table, d)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de l'insertion des données")
        print(e)
        return
    cursor.close()
    print("Les données ont été insérées avec succès")

---

### Importation données table "movies"

_Première table : "movies", essentielle puisque plusieurs clés étrangères sont réliées vers elle_

---

In [37]:
movies_dict = utils.lire_csv_dict(chemin_movies)

In [38]:
sql_inserer_movies_dict = """
    INSERT INTO movies
    (movieId, title, genres)
    VALUES (%(movieId)s, %(title)s, %(genres)s);
"""

In [39]:
inserer_donnees(conn, sql_inserer_movies_dict, movies_dict)

Les données ont été insérées avec succès


---

### Importation données table "ratings"

_Seconde table : "ratings", importante elle aussi car reliées liée particulièrement à "movies"_

---

In [40]:
ratings_dict = utils.lire_csv_dict(chemin_ratings)

In [41]:
sql_inserer_ratings_dict = """
    INSERT INTO ratings
    (userId, movieId, rating, timestamp)
    VALUES (%(userId)s, %(movieId)s, %(rating)s, %(timestamp)s);
"""

In [42]:
inserer_donnees(conn, sql_inserer_ratings_dict, ratings_dict)

Les données ont été insérées avec succès


---

### Importation données table "links"

_Troisième table : "links", la moins instructive des quatre_

---

In [43]:
links_dict = utils.lire_csv_dict(chemin_links)

In [44]:
sql_inserer_links_dict = """
    INSERT INTO links
    (movieId, imdbId, tmdbId)
    VALUES (%(movieId)s, %(imdbId)s, %(tmdbId)s);
"""

In [45]:
inserer_donnees(conn, sql_inserer_links_dict, links_dict)

Les données ont été insérées avec succès


---

### Importation données table "tags"

_Enfin, quatrième et dernière table : "tags"_

---

In [46]:
tags_dict = utils.lire_csv_dict(chemin_tags)

In [47]:
sql_inserer_tags_dict = """
    INSERT INTO tags
    (userId, movieId, tag, timestamp)
    VALUES (%(userId)s, %(movieId)s, %(tag)s, %(timestamp)s);
"""

In [49]:
inserer_donnees(conn, sql_inserer_tags_dict, tags_dict)

Les données ont été insérées avec succès


---

## Lecture des données importées

_Une fois les données importées, place aux requêtes SQL_

---

In [52]:
def lire_donnees(conn, sql_requete):
    try:
        cursor = conn.cursor()
        cursor.execute(sql_requete)
        conn.commit()
    except psycopg2.Error as e:
        print("Erreur lors de la lecture des données")
        print(e)
        return None
    
    print("Les données ont été lues avec succès")
    data = []
    for row in cursor:
        data.append(row)

    cursor.close()
    
    return data

---

### _Quels sont les 15 films les mieux notés ?_

In [61]:
sql_lire_films_notes = """
    SELECT title, count(rating), avg(rating)
    FROM movies INNER JOIN ratings on movies.movieId=ratings.movieId
    GROUP BY title
    HAVING count(rating) > 40
    ORDER BY avg(rating) DESC
    LIMIT 15
"""

In [64]:
lire_donnees(conn, sql_lire_films_notes)

Les données ont été lues avec succès


[('Shawshank Redemption, The (1994)', 317, 4.429022082018927),
 ('Lawrence of Arabia (1962)', 45, 4.3),
 ('Godfather, The (1972)', 192, 4.2890625),
 ('Fight Club (1999)', 218, 4.272935779816514),
 ('Cool Hand Luke (1967)', 57, 4.271929824561403),
 ('Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
  97,
  4.268041237113402),
 ('Rear Window (1954)', 84, 4.261904761904762),
 ('Godfather: Part II, The (1974)', 129, 4.25968992248062),
 ('Departed, The (2006)', 107, 4.252336448598131),
 ('Goodfellas (1990)', 126, 4.25),
 ('Casablanca (1942)', 100, 4.24),
 ('Dark Knight, The (2008)', 149, 4.238255033557047),
 ('Usual Suspects, The (1995)', 204, 4.237745098039215),
 ('Princess Bride, The (1987)', 142, 4.232394366197183),
 ('Star Wars: Episode IV - A New Hope (1977)', 251, 4.231075697211155)]

---

### _Les 15 films les mieux notés avec += 40 avis ?_

In [74]:
sql_lire_films_best = """
    SELECT title, count(rating), avg(rating)
    FROM movies INNER JOIN ratings on movies.movieId=ratings.movieId
    GROUP BY title
    ORDER BY avg(rating) DESC
    LIMIT 15
"""

In [75]:
lire_donnees(conn, sql_lire_films_best)

Les données ont été lues avec succès


[('Mystery of the Third Planet, The (Tayna tretey planety) (1981)', 1, 5.0),
 ('Garden of Words, The (Koto no ha no niwa) (2013)', 1, 5.0),
 ('Brother (Brat) (1997)', 1, 5.0),
 ('Into the Woods (1991)', 1, 5.0),
 ('Dream of Light (a.k.a. Quince Tree Sun, The) (Sol del membrillo, El) (1992)',
  1,
  5.0),
 ('Meantime (1984)', 1, 5.0),
 ('Nasu: Summer in Andalusia (2003)', 1, 5.0),
 ('Holy Motors (2012)', 1, 5.0),
 ('Watching the Detectives (2007)', 1, 5.0),
 ('Eva (2011)', 1, 5.0),
 ('Story of Women (Affaire de femmes, Une) (1988)', 1, 5.0),
 ('Connections (1978)', 1, 5.0),
 ('Go for Zucker! (Alles auf Zucker!) (2004)', 1, 5.0),
 ('Girls About Town (1931)', 1, 5.0),
 ('What Men Talk About (2010)', 1, 5.0)]

---

### _Les utilisateurs les plus prolifiques et note moyenne ?_

In [81]:
sql_lire_user_rating = """
    SELECT userId, COUNT(userId) as c_user, avg(rating)
    FROM ratings
    GROUP BY userId
    ORDER BY c_user DESC
    LIMIT 15
"""

In [82]:
lire_donnees(conn, sql_lire_user_rating)

Les données ont été lues avec succès


[(414, 2698, 3.391957005189029),
 (599, 2478, 2.6420500403551253),
 (474, 2108, 3.398956356736243),
 (448, 1864, 2.8473712446351933),
 (274, 1346, 3.235884101040119),
 (610, 1302, 3.6885560675883258),
 (68, 1260, 3.233730158730159),
 (380, 1218, 3.6732348111658455),
 (606, 1115, 3.6573991031390136),
 (288, 1055, 3.1459715639810426),
 (249, 1046, 3.6964627151051626),
 (387, 1027, 3.2585199610516065),
 (182, 977, 3.5112589559877176),
 (307, 975, 2.6656410256410257),
 (603, 943, 3.5079533404029695)]

---

### _Films avec le plus de tags ?_

In [114]:
sql_lire_count_tags = """
    SELECT title, COUNT(tag) as c_user
    FROM tags INNER JOIN movies on tags.movieId=movies.movieId
    GROUP BY title
    ORDER BY c_user DESC
    LIMIT 15
"""

In [115]:
lire_donnees(conn, sql_lire_count_tags)

Les données ont été lues avec succès


[('Pulp Fiction (1994)', 181),
 ('Fight Club (1999)', 54),
 ('2001: A Space Odyssey (1968)', 41),
 ('Léon: The Professional (a.k.a. The Professional) (Léon) (1994)', 35),
 ('Eternal Sunshine of the Spotless Mind (2004)', 34),
 ('Big Lebowski, The (1998)', 32),
 ('Donnie Darko (2001)', 29),
 ('Star Wars: Episode IV - A New Hope (1977)', 26),
 ('Inception (2010)', 26),
 ('Suicide Squad (2016)', 19),
 ('In the Mood For Love (Fa yeung nin wa) (2000)', 18),
 ('Avatar (2009)', 18),
 ('Pi (1998)', 17),
 ('Eraserhead (1977)', 17),
 ('Avengers: Infinity War - Part I (2018)', 15)]

---

### _25 tags les plus souvent utilisés ?_

In [104]:
sql_lire_tags = """
    SELECT tag, COUNT(tag)
    FROM tags
    GROUP BY tag
    ORDER BY COUNT(tag) DESC
    LIMIT 15
"""

In [105]:
lire_donnees(conn, sql_lire_tags)

Les données ont été lues avec succès


[('In Netflix queue', 131),
 ('atmospheric', 36),
 ('thought-provoking', 24),
 ('superhero', 24),
 ('surreal', 23),
 ('Disney', 23),
 ('funny', 23),
 ('religion', 22),
 ('dark comedy', 21),
 ('sci-fi', 21),
 ('psychology', 21),
 ('quirky', 21),
 ('suspense', 20),
 ('crime', 19),
 ('twist ending', 19)]

---

## Ajouter colonne "year" dans la table "movies"

_Dans une idée d'amélioration de la BBD, création de la colonne "year" dans la table "movies" avant de séparer les données_

---

In [ ]:
def execute_sql(conn, sql_requete):
    
    cursor = conn.cursor()
    cursor.execute(sql_requete)
    conn.commit()
    cursor.close()

In [ ]:
sql_create_year = """
    ALTER TABLE movies
    ADD year VARCHAR
"""

In [ ]:
execute_sql(conn, sql_create_year)

---